In [1]:
# Cell 1 — load + preprocess + build html_content (NON scrive su disco)

import math
import json
from pathlib import Path

import pandas as pd

from const import DATA_DIR

# ==============================================================================
# 1. CONFIGURAZIONE
# ==============================================================================
excel_path = DATA_DIR / "power-plants" / "Global-Nuclear-Power-Tracker-September-2025.xlsx"

# Mappa Colori Ufficiale
color_map = {
    "Operating": "#76c66d",        # Verde
    "Construction": "#ffeb3b",     # Giallo
    "Pre-construction": "#f58231", # Arancio
    "Announced": "#e6194b",        # Rosso
    "Shelved": "#8cb4d6",          # Azzurro chiaro
    "Mothballed": "#4363d8",       # Blu
    "Retired": "#666666",          # Grigio scuro
    "Cancelled": "#bfbfbf"         # Grigio chiaro
}

status_priority = [
    "Operating", "Construction", "Pre-construction", "Announced",
    "Mothballed", "Shelved", "Retired", "Cancelled",
]

print("--- MAP GENERATOR START (3D GLOBE + PIE MARKERS) ---")
print(f"Using Excel: {excel_path}")

# ==============================================================================
# 2. CARICAMENTO DATI
# ==============================================================================
try:
    print("Reading Excel (sheet=Data)...")
    df = pd.read_excel(excel_path, sheet_name="Data")
except Exception as e:
    print(f"ERROR: failed reading input data. {e}")
    raise

# ==============================================================================
# 3. PULIZIA DATI
# ==============================================================================
df = df.dropna(subset=["Latitude", "Longitude"]).copy()

# normalize Status
status_norm_map = {
    "cancelled - inferred 4 y": "cancelled",
    "shelved - inferred 2 y": "shelved",
    "pre construction": "pre-construction",
    "pre-construction": "pre-construction",
}
df["Status"] = (
    df["Status"]
    .astype(str)
    .str.strip()
    .str.lower()
    .replace(status_norm_map)
    .str.replace(r"\s+", " ", regex=True)
    .str.replace("pre-construction", "Pre-construction", regex=False)
    .str.capitalize()
)
# keep exact casing for "Pre-construction"
df.loc[df["Status"].str.lower().eq("pre-construction"), "Status"] = "Pre-construction"

df["Capacity (MW)"] = pd.to_numeric(df.get("Capacity (MW)"), errors="coerce").fillna(0)

# ==============================================================================
# 4. ELABORAZIONE (AGGREGAZIONE PER IMPIANTO)
# ==============================================================================
print("Processing and aggregating per plant...")

plants = []
grouped = df.groupby(["Project Name", "Country/Area", "Region"], dropna=False)

for key, group in grouped:
    lat = float(pd.to_numeric(group["Latitude"], errors="coerce").mean())
    lon = float(pd.to_numeric(group["Longitude"], errors="coerce").mean())
    total_mw = float(pd.to_numeric(group["Capacity (MW)"], errors="coerce").fillna(0).sum())

    operator_col = "Operator"
    operator = "N/A"
    if operator_col in group.columns:
        first_op = group[operator_col].iloc[0]
        operator = first_op if pd.notna(first_op) else "N/A"

    status_counts = group["Status"].value_counts(dropna=False)
    total_units = int(len(group))
    if total_units <= 0 or pd.isna(lat) or pd.isna(lon):
        continue

    sorted_keys = sorted(
        [s for s in status_counts.index.tolist() if isinstance(s, str)],
        key=lambda x: status_priority.index(x) if x in status_priority else 99,
    )

    pie_segments = []
    current_angle = 0.0
    breakdown_html = ""
    statuses_present = []

    for status in sorted_keys:
        count = int(status_counts.get(status, 0))
        if count <= 0:
            continue

        statuses_present.append(status)
        color = color_map.get(status, "#999999")
        percentage = (count / total_units) * 100.0

        pie_segments.append(
            f"{color} {current_angle:.1f}% {current_angle + percentage:.1f}%"
        )
        current_angle += percentage

        mw_sum = float(
            pd.to_numeric(group.loc[group["Status"] == status, "Capacity (MW)"], errors="coerce")
            .fillna(0)
            .sum()
        )
        dot = (
            f'<span style="display:inline-block;width:8px;height:8px;background:{color};'
            'border-radius:50%;margin-right:5px;"></span>'
        )
        breakdown_html += (
            f'<div style="margin-top:2px;">{dot}<b>{status}</b>: '
            f'{count} ({mw_sum:,.0f} MW)</div>'
        )

    gradient_css = f"conic-gradient({', '.join(pie_segments)})" if pie_segments else "#999999"
    size_px = max(10, min(34, int(math.sqrt(max(total_mw, 1.0)) * 0.55)))

    plants.append(
        {
            "name": key[0],
            "country": key[1],
            "region": key[2],
            "lat": lat,
            "lon": lon,
            "total_mw": total_mw,
            "operator": operator,
            "breakdown": breakdown_html,
            "statuses": statuses_present,
            "gradient": gradient_css,
            "size_px": size_px,
        }
    )

regions_list = sorted({p["region"] for p in plants if p.get("region") is not None})
countries_list = sorted({p["country"] for p in plants if p.get("country") is not None})

data_js = json.dumps(plants, ensure_ascii=False)
regions_js = json.dumps(regions_list, ensure_ascii=False)
countries_js = json.dumps(countries_list, ensure_ascii=False)
statuses_js = json.dumps(status_priority, ensure_ascii=False)
colors_js = json.dumps(color_map, ensure_ascii=False)

# ==============================================================================
# 5. HTML TEMPLATE (non f-string: contiene {} e ${} JS)
# ==============================================================================
html_template = """<!DOCTYPE html>
<html lang="en">
<head>
  <meta charset="UTF-8">
  <meta name="viewport" content="width=device-width, initial-scale=1.0">
  <title>Global Nuclear Power Tracker - 3D Globe</title>

  <!-- 3D Globe (Three.js + Globe.gl) -->
  <script src="https://unpkg.com/three@0.160.0/build/three.min.js"></script>
  <script src="https://unpkg.com/globe.gl"></script>

  <!-- 2D Map (MapLibre) -->
  <script src="https://unpkg.com/maplibre-gl@4.7.1/dist/maplibre-gl.js"></script>
  <link href="https://unpkg.com/maplibre-gl@4.7.1/dist/maplibre-gl.css" rel="stylesheet" />

  <style>
    body, html { margin: 0; padding: 0; height: 100%; overflow: hidden; font-family: 'Segoe UI', Arial, sans-serif; background: #000; }
    #globeWrap, #mapWrap { position: absolute; inset: 0; z-index: 0; }
    #mapWrap { display: none; } /* start in 3D */
    #globe { width: 100%; height: 100%; }
    #map { width: 100%; height: 100%; }

    .controls {
      position: fixed; top: 20px; left: 20px; width: 270px; max-height: 90vh;
      background: rgba(255, 255, 255, 0.95); padding: 18px; border-radius: 12px;
      box-shadow: 0 10px 30px rgba(0,0,0,0.5); overflow-y: auto;
      z-index: 100000; /* keep above globe/map markers */
      display: flex; flex-direction: column; gap: 10px;
    }
    h2 { margin: 0 0 5px 0; font-size: 20px; color: #1a1a1a; }
    label { font-size: 12px; font-weight: 600; color: #555; margin-top: 8px; }
    select { width: 100%; padding: 8px; border: 1px solid #ccc; border-radius: 6px; }

    .toggle-btn {
      background: linear-gradient(135deg, #1e3c72 0%, #2a5298 100%);
      color: white; border: none; padding: 14px; border-radius: 8px; font-weight: bold; cursor: pointer;
      transition: all 0.2s; text-align: center; box-shadow: 0 4px 10px rgba(0,0,0,0.3);
      text-transform: uppercase; font-size: 12px; letter-spacing: 0.5px;
    }
    .toggle-btn:hover { transform: scale(1.02); box-shadow: 0 6px 15px rgba(0,0,0,0.4); }

    .status-list { display: flex; flex-direction: column; gap: 4px; max-height: 250px; overflow-y: auto; border: 1px solid #eee; padding: 8px; border-radius: 6px; background: #fafafa; }
    .status-item { display: flex; align-items: center; font-size: 13px; }
    .dot { width: 12px; height: 12px; border-radius: 50%; margin-right: 8px; border: 1px solid rgba(0,0,0,0.1); }

    /* PIE MARKERS (used in both 3D and 2D) */
    .pie-marker {
      border-radius: 50%;
      border: 2px solid #fff;
      box-shadow: 0 2px 6px rgba(0,0,0,0.35);
      cursor: pointer;
      pointer-events: auto;
      touch-action: none;
      will-change: transform;
      transform: translateZ(0);
      z-index: 1;
    }
    .pie-marker:hover { transform: scale(1.15); border-color: #222; }

    /* Globe.gl: keep drag on canvas; markers clickable */
    #globe canvas { pointer-events: auto; }
    #globe .pie-marker { pointer-events: auto; }

    /* While moving/zooming in 2D: disable marker interactions for smoothness */
    .is-moving .pie-marker { pointer-events: none !important; }

    .maplibregl-popup-content { padding: 16px; border-radius: 8px; font-size: 13px; box-shadow: 0 8px 20px rgba(0,0,0,0.3); min-width: 220px; }

    /* 3D Popup (similar to 2D) */
    #globePopup { position: fixed; z-index: 90000; display: none; pointer-events: auto; background: rgba(255,255,255,0.95); color: #111; border-radius: 10px; padding: 12px 14px; min-width: 240px; max-width: 340px; box-shadow: 0 10px 30px rgba(0,0,0,0.45); }
    #globePopup.arrow-left::after, #globePopup.arrow-right::after {
      content: ''; position: absolute; top: var(--arrow-y, 26px); width: 0; height: 0;
      border: 10px solid transparent; transform: translateY(-50%);
    }
    #globePopup.arrow-left::after { left: -20px; border-right-color: rgba(255,255,255,0.95); }
    #globePopup.arrow-right::after { right: -20px; border-left-color: rgba(255,255,255,0.95); }
    #globePopup .close { position: absolute; top: 6px; right: 10px; cursor: pointer; font-weight: 700; color: #444; }
    #globePopup .close:hover { color: #000; }

    .pop-header { font-weight: 700; font-size: 15px; margin-bottom: 4px; border-bottom: 1px solid #eee; padding-bottom: 4px; }
    .pop-meta { color: #777; font-size: 11px; margin-bottom: 8px; }
  </style>
</head>
<body>
  <div id="globeWrap"><div id="globe"></div></div>
  <div id="mapWrap"><div id="map"></div></div>
  <div id="globePopup"></div>

  <div class="controls">
    <h2>Nuclear Power Tracker</h2>
    <div style="font-size:12px; color:#666; margin-bottom:10px;">
      Points are mini pie charts (status mix) and marker size depends on total capacity (MW).
    </div>
    <button class="toggle-btn" id="btnToggle" onclick="toggleView()">Switch to 2D (Flat Map)</button>

    <label>Filter by Region</label>
    <select id="selRegion" onchange="applyFilters()"><option value="all">All regions</option></select>

    <label>Filter by Country</label>
    <select id="selCountry" onchange="applyFilters()"><option value="all">All countries</option></select>

    <label>Plant Status</label>
    <div id="statusList" class="status-list"></div>

    <div style="font-size:11px; color:#888; margin-top:15px; text-align:center; padding-top:10px; border-top:1px solid #eee;">
      <b>3D navigation:</b><br>Drag to rotate the globe.<br>Zoom with wheel/pinch.
    </div>
  </div>

  <script>
    // --- DATA ---
    const plants = __PLANTS__;
    const regions = __REGIONS__;
    const countries = __COUNTRIES__;
    const statuses = __STATUSES__;
    const colors = __COLORS__;

    // --- UI ---
    const selRegion = document.getElementById('selRegion');
    const selCountry = document.getElementById('selCountry');
    const statusList = document.getElementById('statusList');
    regions.forEach(r => selRegion.add(new Option(r, r)));
    countries.forEach(c => selCountry.add(new Option(c, c)));
    statuses.forEach(s => {
      const div = document.createElement('div');
      div.className = 'status-item';
      div.innerHTML = `
        <input type="checkbox" checked value="${s}" onchange="applyFilters()">
        <span class="dot" style="background:${colors[s]}"></span> ${s}
      `;
      statusList.appendChild(div);
    });

    function getCheckedStatuses() {
      return Array.from(document.querySelectorAll('#statusList input:checked')).map(cb => cb.value);
    }

    function tooltipText(p) {
      const mw = Number(p.total_mw || 0).toLocaleString();
      return `${p.name}\n${p.country} | ${p.region}\n${mw} MW`;
    }

    // --- 3D POPUP ---
    const globePopup = document.getElementById('globePopup');
    window.hideGlobePopup = function() {
      if (!globePopup) return;
      globePopup.style.display = 'none';
    };

    function popupHTML(p) {
      return `
        <div class="pop-header">${p.name}</div>
        <div class="pop-meta">${p.country} | ${p.region}</div>
        <div><b>${Number(p.total_mw).toLocaleString()} MW</b></div>
        <hr style="margin:8px 0; border:0; border-top:1px solid #eee;">
        ${p.breakdown || ''}
        <div style="font-size:10px; color:#999; margin-top:8px;">${p.operator || ''}</div>
      `;
    }

    function showGlobePopup(p, anchorEl) {
      if (!globePopup || !anchorEl) return;
      globePopup.innerHTML = `<div class="close" onclick="hideGlobePopup()">×</div>` + popupHTML(p);
      globePopup.style.display = 'block';
      const rect = anchorEl.getBoundingClientRect();
      const pad = 10;
      let x = rect.right + 12;
      let y = rect.top - 12;
      const w = globePopup.offsetWidth || 320;
      const h = globePopup.offsetHeight || 200;
      x = Math.min(window.innerWidth - w - pad, Math.max(pad, x));
      y = Math.min(window.innerHeight - h - pad, Math.max(pad, y));
      globePopup.style.left = x + 'px';
      globePopup.style.top = y + 'px';
      try {
        globePopup.classList.remove('arrow-left', 'arrow-right');
        const markerCx = rect.left + rect.width / 2;
        const markerCy = rect.top + rect.height / 2;
        const isPopupRightOfMarker = x >= (rect.right - 1);
        const isPopupLeftOfMarker = (x + w) <= (rect.left + 1);
        if (isPopupRightOfMarker) globePopup.classList.add('arrow-left');
        else if (isPopupLeftOfMarker) globePopup.classList.add('arrow-right');
        else globePopup.classList.add(markerCx < (x + w / 2) ? 'arrow-left' : 'arrow-right');
        const arrowY = Math.min(h - 18, Math.max(18, markerCy - y));
        globePopup.style.setProperty('--arrow-y', arrowY + 'px');
      } catch(e) {}
    }

    function fixGlobeHtmlLayerPointerEvents() {
      const canvas = globeEl.querySelector('canvas');
      if (canvas) { try { canvas.style.pointerEvents = 'auto'; } catch(e) {} }
      const markers = globeEl.querySelectorAll('.pie-marker');
      if (!markers || markers.length === 0) return;
      markers.forEach(m => { try { m.style.pointerEvents = 'auto'; } catch(e) {} });
      let p = markers[0].parentElement;
      while (p && p !== globeEl) {
        try { p.style.pointerEvents = 'none'; } catch(e) {}
        p = p.parentElement;
      }
    }

    // --- 3D GLOBE ---
    const globeEl = document.getElementById('globe');
    const globe = Globe()(globeEl)
      .globeImageUrl('https://unpkg.com/three-globe/example/img/earth-blue-marble.jpg')
      .bumpImageUrl('https://unpkg.com/three-globe/example/img/earth-topology.png')
      .backgroundColor('#000000')
      .atmosphereColor('#9ecbff')
      .atmosphereAltitude(0.18)
      .htmlLat(d => d.lat)
      .htmlLng(d => d.lon)
      .htmlAltitude(0.01)
      .htmlElement(d => {
        const el = document.createElement('div');
        el.className = 'pie-marker';
        const size = Math.max(10, Math.min(40, Number(d.size_px || 12)));
        el.style.width = size + 'px';
        el.style.height = size + 'px';
        el.style.background = d.gradient || '#999';
        el.title = tooltipText(d);

        let down = null;
        el.addEventListener('contextmenu', (ev) => { ev.preventDefault(); }, { passive: false });
        el.addEventListener('pointerdown', (ev) => {
          if (ev.button !== 0) return;
          down = { x: ev.clientX, y: ev.clientY, t: performance.now() };
          ev.preventDefault();
          ev.stopPropagation();
        }, { passive: false });
        el.addEventListener('pointerup', (ev) => {
          if (ev.button !== 0) return;
          if (!down) return;
          const dx = Math.abs(ev.clientX - down.x);
          const dy = Math.abs(ev.clientY - down.y);
          const dt = performance.now() - down.t;
          down = null;
          if (dx <= 5 && dy <= 5 && dt <= 600) {
            ev.preventDefault();
            ev.stopPropagation();
            showGlobePopup(d, el);
          }
        }, { passive: false });
        el.addEventListener('click', (ev) => { ev.stopPropagation(); }, { capture: true, passive: true });
        return el;
      });

    // --- AUTO-ROTATION ---
    const controls3d = globe.controls();
    function setAutoRotate(on) {
      try { controls3d.autoRotate = !!on; } catch(e) {}
    }
    function configureAutoRotate() {
      try {
        controls3d.enableDamping = true;
        controls3d.dampingFactor = 0.06;
        controls3d.autoRotateSpeed = 1; // slow
      } catch(e) {}
    }
    configureAutoRotate();
    setAutoRotate(true);

    const AUTO_THRESH = 7;
    let down3d = null;
    let moved3d = false;
    let multiTouch3d = false;
    let wheelUsed3d = false;
    const activePointers3d = new Map();

    globeEl.addEventListener('wheel', () => { wheelUsed3d = true; setAutoRotate(false); }, { passive: true });

    globeEl.addEventListener('pointerdown', (ev) => {
      wheelUsed3d = false;
      moved3d = false;
      multiTouch3d = false;
      activePointers3d.set(ev.pointerId, { x: ev.clientX, y: ev.clientY });
      if (activePointers3d.size > 1) multiTouch3d = true;
      down3d = { id: ev.pointerId, x: ev.clientX, y: ev.clientY };
      setAutoRotate(false);
    }, { capture: true, passive: true });

    globeEl.addEventListener('pointermove', (ev) => {
      if (!down3d) return;
      if (activePointers3d.size > 1) multiTouch3d = true;
      const dx = ev.clientX - down3d.x;
      const dy = ev.clientY - down3d.y;
      if (!moved3d && (Math.abs(dx) + Math.abs(dy)) >= AUTO_THRESH) moved3d = true;
    }, { passive: true });

    globeEl.addEventListener('pointerup', (ev) => {
      activePointers3d.delete(ev.pointerId);
      if (!down3d || ev.pointerId !== down3d.id) return;
      const shouldResume = moved3d && !multiTouch3d && !wheelUsed3d;
      down3d = null;
      moved3d = false;
      if (shouldResume) setAutoRotate(true);
    }, { passive: true });

    globeEl.addEventListener('pointercancel', (ev) => {
      activePointers3d.delete(ev.pointerId);
      down3d = null; moved3d = false;
    }, { passive: true });

    globeEl.addEventListener('click', (ev) => {
      try {
        const t = ev && ev.target;
        if (t && t.closest && t.closest('.pie-marker')) return;
      } catch(e) {}
      hideGlobePopup();
    }, { passive: true });

    globe.pointOfView({ lat: 20, lng: 20, altitude: 2.2 }, 0);

    // --- 2D MAP ---
    const map = new maplibregl.Map({
      container: 'map',
      style: {
        version: 8,
        sources: {
          'osm': {
            type: 'raster',
            tiles: ['https://tile.openstreetmap.org/{z}/{x}/{y}.png'],
            tileSize: 256,
            attribution: '&copy; OpenStreetMap'
          }
        },
        layers: [
          { id: 'background', type: 'background', paint: { 'background-color': '#0a0a0a' } },
          { id: 'osm-tiles', type: 'raster', source: 'osm', minzoom: 0, maxzoom: 19 }
        ]
      },
      center: [20, 20],
      zoom: 1.6,
      maxZoom: 7.5,
      projection: 'mercator'
    });
    map.addControl(new maplibregl.NavigationControl(), 'top-right');

    const mapPopup = new maplibregl.Popup({ offset: 14, closeButton: true, closeOnClick: true });
    const mapMarkers = [];

    function buildMapMarkers(data) {
      while (mapMarkers.length) {
        try { mapMarkers.pop().m.remove(); } catch(e) {}
      }
      data.forEach(p => {
        const el = document.createElement('div');
        el.className = 'pie-marker';
        const size = Math.max(10, Math.min(40, Number(p.size_px || 12)));
        el.style.width = size + 'px';
        el.style.height = size + 'px';
        el.style.background = p.gradient || '#999';
        el.title = tooltipText(p);

        el.addEventListener('click', (ev) => {
          ev.stopPropagation();
          mapPopup
            .setLngLat([p.lon, p.lat])
            .setHTML(`
              <div class="pop-header">${p.name}</div>
              <div class="pop-meta">${p.country} | ${p.region}</div>
              <div><b>${Number(p.total_mw).toLocaleString()} MW</b></div>
              <hr style="margin:8px 0; border:0; border-top:1px solid #eee;">
              ${p.breakdown || ''}
              <div style="font-size:10px; color:#999; margin-top:8px;">${p.operator || ''}</div>
            `)
            .addTo(map);
        }, { passive: true });

        const marker = new maplibregl.Marker({ element: el })
          .setLngLat([p.lon, p.lat])
          .addTo(map);

        mapMarkers.push({ m: marker, el: el, data: p });
      });
    }

    // --- FILTERS (both views) ---
    function computeFiltered() {
      const rVal = selRegion.value;
      const cVal = selCountry.value;
      const checked = getCheckedStatuses();
      return plants.filter(p => {
        const visRegion = (rVal === 'all' || p.region === rVal);
        const visCountry = (cVal === 'all' || p.country === cVal);
        const visStatus = (p.statuses || []).some(s => checked.includes(s));
        return visRegion && visCountry && visStatus;
      });
    }

    window.applyFilters = function() {
      const filtered = computeFiltered();
      hideGlobePopup();
      globe.htmlElementsData(filtered);
      setTimeout(fixGlobeHtmlLayerPointerEvents, 0);
      setTimeout(fixGlobeHtmlLayerPointerEvents, 250);
      buildMapMarkers(filtered);
      try { mapPopup.remove(); } catch(e) {}
    };

    // --- TOGGLE VIEW ---
    let currentView = 'globe';
    const globeWrap = document.getElementById('globeWrap');
    const mapWrap = document.getElementById('mapWrap');

    function resizeAll() {
      try { map.resize(); } catch(e) {}
    }

    window.toggleView = function() {
      const btn = document.getElementById('btnToggle');
      if (currentView === 'globe') {
        currentView = 'map';
        globeWrap.style.display = 'none';
        mapWrap.style.display = 'block';
        hideGlobePopup();
        try { setAutoRotate(false); } catch(e) {}
        btn.innerText = 'Switch to 3D (Globe)';
        setTimeout(() => { resizeAll(); }, 50);
      } else {
        currentView = 'globe';
        mapWrap.style.display = 'none';
        globeWrap.style.display = 'block';
        hideGlobePopup();
        try { setAutoRotate(true); } catch(e) {}
        btn.innerText = 'Switch to 2D (Flat Map)';
        setTimeout(() => { resizeAll(); }, 50);
      }
    };

    // smoother 2D while moving/zooming
    const canvasContainer = map.getCanvasContainer();
    map.on('movestart', () => { canvasContainer.classList.add('is-moving'); });
    map.on('moveend', () => { canvasContainer.classList.remove('is-moving'); });

    window.addEventListener('resize', () => resizeAll(), { passive: true });

    // Init
    applyFilters();
  </script>
</body>
</html>"""

html_content = (
    html_template
    .replace("__PLANTS__", data_js)
    .replace("__REGIONS__", regions_js)
    .replace("__COUNTRIES__", countries_js)
    .replace("__STATUSES__", statuses_js)
    .replace("__COLORS__", colors_js)
)

print(f"HTML ready in memory. Aggregated plants: {len(plants)}")


--- MAP GENERATOR START (3D GLOBE + PIE MARKERS) ---
Using Excel: C:\Users\Fede\OneDrive - SUPSI\Secondo_anno_bachelor\Data_Visualization\Project\nuclear-footprints\data\power-plants\Global-Nuclear-Power-Tracker-September-2025.xlsx
Reading Excel (sheet=Data)...
Processing and aggregating per plant...
HTML ready in memory. Aggregated plants: 571


In [2]:
from const import VISUALIZATIONS_DIR

output_html_path = VISUALIZATIONS_DIR / "global-nuclear-power-tracker.html"
output_html_path.write_text(html_content, encoding="utf-8")

329657